In [150]:
import pandas as pd
import sys
import sqlalchemy
import os


%matplotlib inline

CONN = sqlalchemy.create_engine('sqlite:///data.db')


#CONN.execute('ALTER TABLE player_stats ADD COLUMN position TEXT;')
CONN.execute('PRAGMA table_info(player_stats)').fetchall()
#CONN.execute(sqlalchemy.text('SELECT * from player_stats where player_id = :p '),p=101106).fetchone()


[(0, 'index', 'BIGINT', 0, None, 0),
 (1, 'player_id', 'BIGINT', 0, None, 0),
 (2, 'full_name', 'TEXT', 0, None, 0),
 (3, 'first_name', 'TEXT', 0, None, 0),
 (4, 'last_name', 'TEXT', 0, None, 0),
 (5, 'team', 'TEXT', 0, None, 0),
 (6, 'games_played', 'BIGINT', 0, None, 0),
 (7, 'minutes', 'FLOAT', 0, None, 0),
 (8, 'pull_up_shots_points_per_game', 'FLOAT', 0, None, 0),
 (9, 'pull_up_field_goals_made_per_game', 'FLOAT', 0, None, 0),
 (10, 'pull_up_field_goals_attempted_per_game', 'FLOAT', 0, None, 0),
 (11, 'pull_up_field_goal_percentage', 'FLOAT', 0, None, 0),
 (12, 'pull_up_3_point_field_goals_made_per_game', 'FLOAT', 0, None, 0),
 (13, 'pull_up_3_point_field_goals_attempted_per_game', 'FLOAT', 0, None, 0),
 (14, 'pull_up_3_point_percentage', 'FLOAT', 0, None, 0),
 (15, 'pull_up_3_point_effective_field_goal_percentage', 'FLOAT', 0, None, 0),
 (16, 'pull_up_points_total', 'BIGINT', 0, None, 0),
 (17, 'drives_per_game', 'FLOAT', 0, None, 0),
 (18, 'points_per_game_off_drives', 'FLOAT', 

In [149]:
os.listdir('datasets/Sports')
CONN.execute(sqlalchemy.text('SELECT position from season_stats s where s.name = "Curly Armstrong" ')).fetchone()


('G-F',)

In [151]:
nba_data_path = 'datasets/Sports/Basketball/Nba Data.csv'
players_path = 'datasets/Sports/Basketball/Players.csv'
season_stats = 'datasets/Sports/Basketball/Seasons_Stats.csv'

nba_df = pd.read_csv(nba_data_path)
players_df = pd.read_csv(players_path)
season_stats_df = pd.read_csv(season_stats)


player_percentages = nba_df[list([column for column in nba_df.columns if ('Percentage' in column and 'Rebound' not in column and 'Effective' not in column and 'Against' not in column) or column == 'Name'])]

player_percentages.to_sql('player_percentages',CONN,if_exists='replace')
players_df.to_sql('player',CONN, if_exists='replace')
season_stats_df.to_sql('season_stats',CONN, if_exists='replace')


players_df.columns = ['id','name','height','weight','college','born','birth_city','birth_state']

columns = list(season_stats_df.columns)
columns[0] = 'id'
season_stats_df.columns = columns


duplicate_players_total = nba_df[(nba_df.duplicated(subset=['Full Name'])) & (nba_df['Team'] == 'TOTAL')]

nba_df = nba_df.drop_duplicates(subset=['Full Name'], keep=False)
season_stats_df = season_stats_df.drop_duplicates(subset=['Name'], keep=False)

nba_player_stats_df = pd.concat([nba_df, duplicate_players_total])

nba_player_stats_cols = list(nba_player_stats_df.columns)
nba_player_stats_cols[32] = 'Second Pass Hockey Assist'
nba_player_stats_cols = ['_'.join(column.strip().lower().split())  for column in nba_player_stats_cols]

nba_player_stats_df.columns = nba_player_stats_cols


nba_player_stats_df.team = nba_player_stats_df.team.apply(lambda x : x.lower())

nbapstatsdfkmeans = nba_player_stats_df[['player_id','games_played',
 'minutes',
 'pull_up_shots_points_per_game',
 'pull_up_field_goals_made_per_game',
 'pull_up_field_goals_attempted_per_game',
 'pull_up_field_goal_percentage',
 'pull_up_3_point_field_goals_made_per_game',
 'pull_up_3_point_field_goals_attempted_per_game',
 'pull_up_3_point_percentage',
 'pull_up_3_point_effective_field_goal_percentage',
 'pull_up_points_total',
 'drives_per_game',
 'points_per_game_off_drives',
 'total_points_per_game_on_drives_(including_ft)',
 'field_goal_percentage_on_drives',
 'points_per_48_minutes_on_drives',
 'total_points_on_drives',
 'total_drives',
 'blocks_per_game',
 'steals_per_game',
 'field_goals_made_against_when_defending_rim',
 'field_goals_attempted_against_when_defending_rim',
 'field_goal_percentage_against_when_defending_rim',
 'total_blocks',
 'passes_per_game',
 'assists_per_game',
 'free_throws_assisted_per_game',
 'second_pass_hockey_assist',
 'assist_opportunities_per_game',
 'points_created_off_assists_per_game',
 'points_created_off_assists_per_game.1',
 'total_assists',
 'touches_per_game',
 'front_court_touches_per_game',
 'time_of_possession',
 'close_range_touches_per_game',
 'elbow_touches_per_game',
 'points_per_game',
 'points_per_touch',
 'points_per_half_court_touch',
 'total_touches',
 'total_distance_(miles)',
 'average_speed',
 'distance_per_game_(miles)',
 'distance_per_48_minutes_(miles)',
 'total_distance_on_offense_(miles)',
 'total_distance_on_defense_(miles)',
 'average_speed_offense',
 'average_speed_defense',
 'rebounds_per_game',
 'rebound_chances',
 'rebound_percentage',
 'contested_rebounds_per_game',
 'uncontested_rebounds_per_game',
 'percentage_of_rebounds_uncontested',
 'total_rebounds',
 'offensive_rebounds_per_game',
 'offensive_rebound_chances',
 'offensive_rebounds_collected_percentage',
 'offensive_rebounds_contested',
 'offensive_rebounds_uncontested',
 'offensive_rebounds_uncontested_percentage',
 'defensive_rebounds',
 'defensive_rebound_chances',
 'defensive_rebounds_collected_percentage',
 'defensive_rebounds_contested',
 'defensive_rebounds_uncontested',
 'defensive_rebounds_uncontested_percentage',
 'catch_and_shoot_points_per_game',
 'catch_and_shoot_field_goal_made_per_game',
 'catch_and_shoot_field_goal_attempted_per_game',
 'catch_and_shoot_field_goal_percentage',
 'catch_and_shoot_3_point_made_per_game',
 'catch_and_shoot_3_point_attempts_per_game',
 'catch_and_shoot_3_point_field_goal_percentage',
 'catch_and_shoot_effective_field_goal_percentage',
 'catch_and_shoot_points_total',
 'points_per_game.1',
 'points_per_game_drive',
 'field_goal_percentage_drive',
 'points_per_game_close_range',
 'field_goal_percentage_close_range',
 'points_per_game_catch_and_shoot',
 'field_goal_percentage_catch_and_shoot',
 'points_per_game_pull_up',
 'field_goal_percentage_pull_up',
 'field_goal_attempted_drive',
 'field_goal_attempted_close_range',
 'field_goal_attempted_catch_and_shoot',
 'field_goal_attempted_pull_up',
 'effective_field_goal_percentage']]

season_stats_df = season_stats_df.query('Year>2013')

season_stats_df.index = pd.RangeIndex(len(season_stats_df.index))
season_stats_df.id = pd.RangeIndex(len(season_stats_df.index))

seasonstats_means = season_stats_df[['id', 'Year', 'Age', 'Games Played',
       'Games Started', 'Minutes Per Game', 'Player Efficiency Rating ',
       'True Shooting Percentage', '3 Point Attempted Rate', 'Free Throw Rate',
       'Offensive Rebound Percentage', 'Defensive Rebound Percentage',
       'True Rebounding Percentage', 'Assist Percentage', 'Steal Percentage',
       'Block Percentage', 'Turn Over Percentage', 'Usage Rate', 'Unnamed: 21',
       'Offensive Win Shares', 'Defensive Win Shares', 'Win Shares',
       'Win Shares Per 48 Minutes', 'Unnamed: 26',
       'Offensive Basketball Plus Minus', 'Defensive Basketball Plus Minus',
       'Basketball Plus Minus', 'Value Over Replacement Player',
       'Field Goals Made', 'Field Goals Attempted', 'Field Goal Percentage',
       'Three Point Field Goals Made', 'Three Point Field Goals Attempted',
       'Three Point Field Goal Percentage', 'Two Point Field Goals Made',
       'Two Point Field Goals Attempted', 'Two Point Field Goal Percentage',
       'Effective Field Goal Percentage', 'Free Throws Made',
       'Free Throws Attempted', 'Free Throw Percentage', 'Offensive Rebounds',
       'Defensive Rebounds', 'Total Rebounds', 'Total Assists', 'Total Steals',
       'Total Blocks', 'Total Turnovers', 'Total Personal Fouls',
       'Total Points']]

season_stats_df["Position"]


,id,name,height,weight,college,born,birth_city,birth_state
0,0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky


In [117]:
# Solving null values problem in df. 
nbapstatsdfkmeans =  nbapstatsdfkmeans.fillna(value=0.0)
seasonstats_means = seasonstats_means.fillna(value=0.0)

seasonstats_means.head()


,id,Year,Age,Games Played,Games Started,Minutes Per Game,Player Efficiency Rating,True Shooting Percentage,3 Point Attempted Rate,Free Throw Rate,...,Free Throw Percentage,Offensive Rebounds,Defensive Rebounds,Total Rebounds,Total Assists,Total Steals,Total Blocks,Total Turnovers,Total Personal Fouls,Total Points
0,0,2014.0,23.0,14.0,0.0,132.0,3.7,0.367,0.900,0.000,...,0.000,4.0,13.0,17.0,3.0,6.0,0.0,3.0,13.0,22.0
1,1,2014.0,27.0,31.0,0.0,198.0,11.1,0.489,0.500,0.387,...,0.750,13.0,25.0,38.0,8.0,2.0,3.0,6.0,19.0,71.0
2,2,2014.0,26.0,2.0,0.0,12.0,-1.0,0.333,0.000,0.000,...,0.000,1.0,1.0,2.0,0.0,0.0,0.0,1.0,1.0,2.0
3,3,2014.0,25.0,37.0,8.0,488.0,11.0,0.502,0.272,0.252,...,0.649,42.0,86.0,128.0,16.0,14.0,27.0,41.0,74.0,164.0
4,4,2014.0,23.0,7.0,0.0,38.0,13.3,0.603,0.357,0.286,...,0.750,1.0,5.0,6.0,4.0,0.0,0.0,4.0,4.0,19.0


In [118]:
from sklearn.cluster import KMeans
import collections

kmeans = KMeans().fit(nbapstatsdfkmeans)

clustering = collections.defaultdict(list)

for idx, label in enumerate(kmeans.labels_):
    clustering[label].append(idx)
 

name_lookup = {record['player_id']:record['full_name'] for record in nba_player_stats_df[['player_id','full_name']].to_dict('records')}


for cluster in clustering:
    print("Cluster: %d"%(cluster))
    player_cluster = nbapstatsdfkmeans[nbapstatsdfkmeans.index.isin(clustering[cluster])][['player_id']]
    player_cluster.name = player_cluster.player_id.apply(lambda x : name_lookup[x])
    print(player_cluster.name)
    


Cluster: 2
0         Andrew Bogut
1      Marvin Williams
2           Chris Paul
3       Raymond Felton
4      Martell Webster
6        Channing Frye
7       Deron Williams
14        Gerald Green
15       Nate Robinson
17    Francisco Garcia
19         Ian Mahinmi
20           David Lee
21        Brandon Bass
23      Ersan Ilyasova
24        Ronny Turiaf
26      Louis Williams
10        Andrew Bynum
Name: player_id, dtype: object
Cluster: 7
5      Charlie Villanueva
16           Jarrett Jack
18          Jason Maxiell
22               CJ Miles
25            Monta Ellis
27         Andray Blatche
28             Ryan Gomes
29           Amir Johnson
30          Marcin Gortat
417          Robert Sacre
418         Ognjen Kuzmic
419       Hollis Thompson
13          Danny Granger
Name: player_id, dtype: object
Cluster: 0
31         Ronnie Price
35           Will Bynum
42      Stephen Jackson
46          Paul Pierce
348       E'Twaun Moore
349       Isaiah Thomas
351        Chris Wright
359     

In [121]:
import matplotlib.pylab as plt
clusters=range(1,20)
meandist=[]

kmeans_stats = KMeans().fit(seasonstats_means)

clustering = collections.defaultdict(list)

for idx, label in enumerate(kmeans_stats.labels_):
    clustering[label].append(idx)

name_lookup_seasons = {record['id']:record['Name'] for record in season_stats_df[['id','Name']].to_dict('records')}
efficiency_lookup_seasons = {record['id']:record['Player Efficiency Rating '] for record in season_stats_df[['id','Player Efficiency Rating ']].to_dict('records')}

# loop through each cluster and fit the model to the train set
# generate the predicted cluster assingment and append the mean distance my taking the sum divided by the shape
for cluster in clustering:
    print("Cluster: %d"%(cluster))
    player_cluster = seasonstats_means[seasonstats_means.index.isin(clustering[cluster])][['id']]
    player_cluster.name = player_cluster.id.apply(lambda x : name_lookup_seasons[x])
    player_cluster.efficiency = player_cluster.id.apply(lambda x : efficiency_lookup_seasons[x])
    print(player_cluster.name+" - "+str(player_cluster.efficiency))


Cluster: 0
0      Chris Babb - 0       3.7\n2      -1.0\n4      ...
2      Shane Edwards - 0       3.7\n2      -1.0\n4   ...
4      Carrick Felix - 0       3.7\n2      -1.0\n4   ...
5      Elias Harris - 0       3.7\n2      -1.0\n4    ...
6      Scotty Hopson - 0       3.7\n2      -1.0\n4   ...
7      Erik Murphy - 0       3.7\n2      -1.0\n4     ...
8      Nemanja Nedovic - 0       3.7\n2      -1.0\n4 ...
10     D.J. Stephens - 0       3.7\n2      -1.0\n4   ...
11     Casper Ware - 0       3.7\n2      -1.0\n4     ...
12     Royce White - 0       3.7\n2      -1.0\n4     ...
14     Jerrelle Benimon - 0       3.7\n2      -1.0\n4...
15     Sim Bhullar - 0       3.7\n2      -1.0\n4     ...
17     Will Cherry - 0       3.7\n2      -1.0\n4     ...
18     Patrick Christopher - 0       3.7\n2      -1.0...
19     Jack Cooley - 0       3.7\n2      -1.0\n4     ...
20     Andre Dawkins - 0       3.7\n2      -1.0\n4   ...
21     Drew Gordon - 0       3.7\n2      -1.0\n4     ...
23     Alex Kirk - 0

In [157]:
season_stats_df.columns

Index(['id', 'Year', 'Name', 'Position', 'Age', 'Team', 'Games Played',
       'Games Started', 'Minutes Per Game', 'Player Efficiency Rating ',
       'True Shooting Percentage', '3 Point Attempted Rate', 'Free Throw Rate',
       'Offensive Rebound Percentage', 'Defensive Rebound Percentage',
       'True Rebounding Percentage', 'Assist Percentage', 'Steal Percentage',
       'Block Percentage', 'Turn Over Percentage', 'Usage Rate', 'Unnamed: 21',
       'Offensive Win Shares', 'Defensive Win Shares', 'Win Shares',
       'Win Shares Per 48 Minutes', 'Unnamed: 26',
       'Offensive Basketball Plus Minus', 'Defensive Basketball Plus Minus',
       'Basketball Plus Minus', 'Value Over Replacement Player',
       'Field Goals Made', 'Field Goals Attempted', 'Field Goal Percentage',
       'Three Point Field Goals Made', 'Three Point Field Goals Attempted',
       'Three Point Field Goal Percentage', 'Two Point Field Goals Made',
       'Two Point Field Goals Attempted', 'Two Point Fiel